In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error



from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases
from decimal import Decimal
import datetime as dt 

In [16]:
df = pd.read_csv('C:/Users/kusha/OneDrive/Desktop/Task/OnlineRetail.csv',encoding='unicode_escape')
#df.drop('Unnamed: 0', inplace=True, axis=1)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [17]:
df.isna().sum() 

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [21]:
df.dropna(inplace=True)

In [23]:

df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [24]:
df=df[~df['Quantity']<0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  object 
 5   UnitPrice    397924 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      397924 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


In [26]:
df['InvoiceNo']=df['InvoiceNo'].astype('str')
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df['CustomerID']=df['CustomerID'].astype('str')
df['Description']=df['Description'].astype('str')


In [29]:
df['StockCode']=df['StockCode'].astype('str')
df['Country']=df['Country'].astype("str")
df['UnitPrice']=df['UnitPrice'].apply(Decimal)

In [32]:
df['Monetary']=df['Quantity']*df['UnitPrice']
df['Monetary']=df['Monetary'].apply(Decimal)

In [33]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Monetary
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.54999999999999982236431605997495353221893310...,17850.0,United Kingdom,15.29999999999999893418589636
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.84999999999999997779553950749686919152736663...,12680.0,France,10.19999999999999973354647409
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10000000000000008881784197001252323389053344...,12680.0,France,12.60000000000000053290705182
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15000000000000035527136788005009293556213378...,12680.0,France,16.60000000000000142108547152
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15000000000000035527136788005009293556213378...,12680.0,France,16.60000000000000142108547152


In [34]:
df_rfmt=summary_data_from_transaction_data(transactions=df,customer_id_col='CustomerID',datetime_col='InvoiceDate',monetary_value_col='Monetary')
df_rfmt.head()

,frequency,recency,T,monetary_value
CustomerID,,,,
12346.0,0.0,0.0,325.0,0.000000
12347.0,6.0,365.0,367.0,599.701667
12348.0,3.0,283.0,358.0,301.480000
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000


In [35]:
diff_time=df['InvoiceDate'].max()-df['InvoiceDate'].min()

In [38]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Monetary
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.54999999999999982236431605997495353221893310...,17850.0,United Kingdom,15.29999999999999893418589636
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000000000000012434497875801753252744674682...,17850.0,United Kingdom,20.34000000000000074606987255
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.84999999999999997779553950749686919152736663...,12680.0,France,10.19999999999999973354647409
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10000000000000008881784197001252323389053344...,12680.0,France,12.60000000000000053290705182
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15000000000000035527136788005009293556213378...,12680.0,France,16.60000000000000142108547152
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15000000000000035527136788005009293556213378...,12680.0,France,16.60000000000000142108547152


In [39]:
diff_time=df['InvoiceDate'].max()-df['InvoiceDate'].min()

In [40]:
diff_time

Timedelta('373 days 04:24:00')

In [41]:
end_date_cal=df['InvoiceDate'].min() +dt.timedelta(days=200)
end_date_obs=end_date_cal+(diff_time-dt.timedelta(days=200))

In [42]:
end_date_cal

Timestamp('2011-06-19 08:26:00')

In [43]:
df_rfmt_cal=calibration_and_holdout_data(transactions=df,customer_id_col='CustomerID',datetime_col='InvoiceDate',calibration_period_end=end_date_cal,observation_period_end=end_date_obs)
df_rfmt_cal

,frequency_cal,recency_cal,T_cal,frequency_holdout,duration_holdout
CustomerID,,,,,
12346.0,0.0,0.0,152.0,0.0,173.0
12347.0,3.0,184.0,194.0,3.0,173.0
12348.0,2.0,110.0,185.0,1.0,173.0
12350.0,0.0,0.0,137.0,0.0,173.0
12352.0,3.0,34.0,123.0,3.0,173.0
...,...,...,...,...,...
18273.0,0.0,0.0,84.0,2.0,173.0
18280.0,0.0,0.0,104.0,0.0,173.0
18281.0,0.0,0.0,7.0,0.0,173.0
